Import and setup some auxiliary functions

In [7]:
import torch
from torchvision import transforms, datasets
import numpy as np
import timeit
from collections import OrderedDict
from pprint import pformat
from torch.utils.data.sampler import *
from tqdm import tqdm
import time
from google.colab import drive
from prettytable import PrettyTable

import torch.nn as nn
import torch.nn.functional as F

torch.multiprocessing.set_sharing_strategy('file_system')

def compute_score(acc, min_thres, max_thres):
    if acc <= min_thres:
        base_score = 0.0
    elif acc >= max_thres:
        base_score = 100.0
    else:
        base_score = float(acc - min_thres) / (max_thres - min_thres) \
                     * 100
    return base_score


def run(algorithm, dataset_name, filename):
    predicted_test_labels, gt_labels, run_time, parameters_count = algorithm(dataset_name)
    if predicted_test_labels is None or gt_labels is None:
      return (0, 0, 0)

    correct = 0
    total = 0
    for label, prediction in zip(gt_labels, predicted_test_labels):
      total += label.size(0)
      correct += (prediction.cpu().numpy() == label.cpu().numpy()).sum().item()   # assuming your model runs on GPU
      
    accuracy = float(correct) / total
    
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
    return (correct, accuracy, run_time, parameters_count)

Your implementation starts here

In [8]:
# Part[1] TODO: Cifar-10 dataloading
import torchvision
from torch.utils.data import random_split
def load_data(dataset_name, device, config):
    """
    loads cifar-10 dataset using torchvision, take the last 5k of the training data to be validation data
    """

    """
    loads cifar-10 dataset using torchvision, take the last 5k of the training data to be validation data
    """

    batch_size_train = 20
    batch_size_test = 1

    transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    CIFAR10_training = torchvision.datasets.CIFAR10(root='./data', train=True,
                                                    download=True, transform=transform)
    CIFAR10_test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                                    download=True, transform=transform)

    # create a training and a validation set
    CIFAR10_training_set, CIFAR10_validation_set = random_split(
        CIFAR10_training, [45000, 5000])

    # Create data loaders
    train_dataloader = torch.utils.data.DataLoader(CIFAR10_training_set,
                                               batch_size=batch_size_train,
                                               shuffle=True, num_workers=2)
    valid_dataloader = torch.utils.data.DataLoader(CIFAR10_validation_set,
                                                    batch_size=batch_size_train,
                                                    shuffle=True, num_workers=2)
    test_dataloader = torch.utils.data.DataLoader(CIFAR10_test_set,
                                              batch_size=batch_size_test,
                                              shuffle=False, num_workers=2)

        
    return train_dataloader, valid_dataloader, test_dataloader

In [9]:
# Part [2] TODO: Main model definition + any utilities such as weight initialization
# Reference https://shonit2096.medium.com/cnn-on-cifar10-data-set-using-pytorch-34be87e09844
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer
        self.conv1 = nn.Conv2d(3, 8, 3, padding=1)
        torch.nn.init.xavier_uniform_(self.conv1.weight)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
        torch.nn.init.xavier_uniform_(self.conv2.weight)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)
        torch.nn.init.xavier_uniform_(self.conv3.weight)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        self.upSampling = nn.UpsamplingNearest2d(scale_factor=2)
        # fully connected layers
        self.fc1 = nn.Linear(32 * 4 * 4, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)
        # dropout
        self.dropout = nn.Dropout(p=.5)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.upSampling(x)
        x = self.pool(x)
        # flattening
        x = x.view(-1, 32 * 4 * 4)
        # fully connected layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

In [10]:
# Part [3] TODO : Main trainig + validation, returns a trained model
import torch.optim as optim
def train(train_dataloader, valid_dataloader, device, config):
    
    n_epochs = 60
    learning_rate = 0.01

    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    def trainA(epoch):
      model.train()
      for batch_idx, (data, target) in enumerate(train_dataloader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)+ 0.005 * torch.sum(
                torch.sum(torch.pow(list(model.parameters())[0], 2)), 0)
        loss.backward()
        optimizer.step()


    def validation(epoch):
      model.eval()
      validation_loss = 0
      correct = 0
      with torch.no_grad():
        for data, target in valid_dataloader:
          data = data.to(device)
          target = target.to(device)
          output = model(data)
          validation_loss += criterion(output, target).item() + 0.005 * torch.sum(
                torch.sum(torch.pow(list(model.parameters())[0], 2)), 0)
          pred = output.data.max(1, keepdim=True)[1]
          correct += pred.eq(target.data.view_as(pred)).sum()
      validation_loss /= len(valid_dataloader.dataset)
      print('Train Epoch: {} Validation set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        epoch, validation_loss, correct, len(valid_dataloader.dataset),
        100. * correct / len(valid_dataloader.dataset)))
    
    validation(0)
    for epoch in range(1, n_epochs + 1):
        trainA(epoch)
        validation(epoch)

    return model
  

In [11]:
# Part [4] TODO: Testing + paramater setting
# TODO: Testing + paramater setting

def save_model_colab_for_submission(model):  # if you are running on colab
  drive.mount('/content/gdrive/', force_remount=True)
  
  torch.save(model.to(torch.device("cpu")), '/content/gdrive/My Drive/model.pt') # you will find the model in your home drive
  

def save_model_local_for_submission(model):  # if you are running on your local machine
  torch.save(model.to(torch.device("cpu")), 'model.pt')
  
def test(model, test_dataloader, device):
  test_predictions = []
  true_labels = []

  criterion = nn.CrossEntropyLoss()

  test_loss = 0
  correct = 0
  count = 0
  model.eval()
  with torch.no_grad():
    for data, target in test_dataloader:
      data = data.to(device)
      target = target.to(device)
      output = model(data)
      test_loss += criterion(output, target).item() + 0.005 * torch.sum(
                torch.sum(torch.pow(list(model.parameters())[0], 2)), 0)
      pred = output.data.max(1, keepdim=True)[1]
      if count == 0:
        preds = pred
        gts = target.data.view_as(pred)
      else:
        preds = torch.cat((preds, pred), 0)
        gts = torch.cat((gts, target.data.view_as(pred)), 0)
      count += 1
      correct += pred.eq(target.data.view_as(pred)).sum()
      test_loss /= len(test_dataloader.dataset)
  
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_dataloader.dataset),
    100. * correct / len(test_dataloader.dataset)))
  
  test_predictions = preds

  true_labels = gts

  return test_predictions, true_labels

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    # uncomment below codes for your debugging
    #print(table)
    #print(f"Total Trainable Params: {total_params}")
    return total_params

def run_NN(dataset_name):
    # set parameters cifar10
  config = {
        'lr': 0,
        'num_epochs': 0,
        'batch_size': 0,
        'num_classes': 0,
        'regular_constant': 0,
        'transforms': transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) }
    
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  
  train_dataloader, valid_dataloader, test_dataloader = load_data(dataset_name, device, config)
  
  model = train(train_dataloader, valid_dataloader, device, config)
  parameters_count = count_parameters(model)

  device = torch.device("cpu")
  start_time = timeit.default_timer()
  assert test_dataloader.batch_size == 1, 'Error: You should use use batch size = 1 for the test loader.'
  preds, labels = test(model.to(device), test_dataloader, device)
  end_time = timeit.default_timer()
  

  test_time = (end_time - start_time)
  print("Total run time of testing the model: ", test_time , " seconds.")
  
  save_model_colab_for_submission(model)
  
  return preds, labels, test_time, parameters_count

Main loop. Run time and total score will be shown below.

In [ ]:
# Don't edit this cell
def run_on_dataset(dataset_name, filename):
    min_thres = 0.55
    max_thres = 0.65

    correct_predict, accuracy, run_time, parameters_count = run(run_NN, dataset_name, filename)

    score = compute_score(accuracy, min_thres, max_thres)
    if parameters_count > 50000:
      score = max(0, score - 25)
    result = OrderedDict(
                  score=score,
                  correct_predict=correct_predict,
                  accuracy=accuracy,
                  run_time=run_time,
                  parameters_count=parameters_count)
    return result, score


def main():
    filenames = { "CIFAR10": "predictions_cifar10_YourName_IDNumber.txt"}
    result_all = OrderedDict()
    scores = []
    for dataset_name in ["CIFAR10"]:
        result_all, this_score = run_on_dataset(dataset_name, filenames[dataset_name])
    with open('result.txt', 'w') as f:
        f.writelines(pformat(result_all, indent=4))
    print("\nResult:\n", pformat(result_all, indent=4))


main()

Files already downloaded and verified
Files already downloaded and verified
Train Epoch: 0 Validation set: Avg. loss: 0.1164, Accuracy: 472/5000 (9%)

